In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=15, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")
n_pop = 10
n_gen = 10
n_sats = 12

In [3]:
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
# stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")
opt = Optimizer(stk_object,n_pop,n_gen,n_sats,weights=(7,-8,-1,-2))

In [4]:
stk_object.Interpolate=True
stk_object.dt = 30
starting_ind = get_ind()                                # Gets individual associating with ../../Input_Files/Satellite_File.txt
hof,percent,std,time,sats = opt.run(starting_ind)       # Run optimization putting individual into the initial population
opt.cost_function(hof[0])                               # Get best individual from optimization and load them into ../../Input_Files/Satellite_File.txt

In [ ]:
stk_object.dt = 2.5
stk_object.Generate_Planning_Data()

In [ ]:
stk_object.Plan_Data(slew_rate=1)

In [ ]:
data_comparison = {}
data_comparison["Unplanned (%)"] = [len(np.unique(stk_object.Planning_Data[stk_object.Planning_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Unplanned (Time)"] = [max(stk_object.Planning_Data[stk_object.Planning_Data['Target'].values==tar_num]['Time'].values/86400) for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values/86400) for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

In [ ]:
fig = px.scatter(stk_object.Planned_Data.sort_values(by="Target"),x="Time",y="Bin Number",hover_data="Bin Number",color="Satellite",animation_frame="Target")
fig.show()